In [73]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Reshape, BatchNormalization,\
                                    Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model, Sequential
import numpy as np
from tensorflow.keras.datasets import mnist

In [74]:
# CONSTANTS
rows, cols = (28, 28)
channels = 1
img_shape = (rows, cols, channels)

# Model

In [93]:
def build_generator():

    noise_shape = (100,)     
    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    noise = Input(shape=noise_shape)
    img = model(noise)    #Generated image

    return Model(noise, img)

In [94]:
generator = build_generator()

In [96]:
def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)


In [97]:
discriminator = build_discriminator()

# Train

In [98]:
optimizer = Adam(0.0002, 0.5)

In [99]:
generator.compile(optimizer=optimizer,loss="binary_crossentropy")
discriminator.compile(optimizer=optimizer,loss="binary_crossentropy", metrics=['accuracy'])
input_noise = Input(shape=(100,))
fake_img = generator(input_noise)
validity = discriminator(fake_img)
combined = Model(input_noise, validity)
combined.compile(optimizer=optimizer, loss="binary_crossentropy")

In [107]:
import os
import matplotlib.pyplot as plt

def save_imgs(epoch):

  root_dir = "/content/drive/MyDrive/DeepLearning_Models/GAN/epochs_image"
  try:
    os.mkdir(root_dir)
  except:
    print("Folder exists")
  
  r,c = 2,2
  noise = np.random.normal(0,1,(r*c, 100))
  gen_imgs = generator.predict(noise)
  gen_imgs = 0.5 * gen_imgs + 0.5

  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
      for j in range(c):
          axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
          axs[i,j].axis('off')
          cnt += 1
  fig.savefig(f"{root_dir}/mnist_{epoch}.png")
  plt.close()
  return

In [112]:
def train(epochs = 10000, batch_size = 128, save_every = 2000):
  (X_train, _), (_, _) = mnist.load_data()
  X_train = (X_train.astype("float") - 127.5) / 127.5   # This is to normalize the image between 0 to 1
  half_batch = int(batch_size / 2)
  X_train = np.expand_dims(X_train, axis=3)

  for epoch in range(1,epochs+1):

    # TRAIN DISCRIMINATOR
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    img = X_train[idx]  # real image from the dataset

    # Noise
    noise = np.random.normal(0, 1, (half_batch, 100)) # input to generator
    fake_image = generator.predict(noise) # fake image from GENERATOR

    # Discriminator training  np.ones-real image np.zeros-fake image
    d_loss_real = discriminator.train_on_batch(img, np.ones((half_batch,1)))
    d_loss_fake = discriminator.train_on_batch(fake_image, np.zeros((half_batch,1)))

    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # TRAIN GENERATOR
    noise = np.random.normal(0, 1, (batch_size, 100))
    y = np.array([1] * batch_size)
    g_loss = combined.train_on_batch(noise, y)

    if((epoch % save_every) == 0):
      print(f"saved {epoch}")
      save_imgs(epoch)



In [113]:
train()

saved 2000
Folder exists
saved 4000
Folder exists
saved 6000
Folder exists
saved 8000
Folder exists
saved 10000
Folder exists


In [116]:
root_dir = "/content/drive/MyDrive/DeepLearning_Models/GAN/"

In [117]:
generator.save(root_dir+"generator.h5")